In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv

In [ ]:


# Hàm thu thập liên kết công việc từ một trang
def get_job_links(driver):
    # Tìm tất cả thẻ <a> chứa liên kết công việc
    elements = driver.find_elements(By.XPATH, "//a[contains(@class, 'job_link')]")
    # Lấy thuộc tính 'href' của từng phần tử
    links = [element.get_attribute("href") for element in elements]
    return links





# Hàm duyệt qua nhiều trang và thu thập liên kết
def scrape_multiple_pages(start_url, max_pages):
    driver = webdriver.Chrome(executable_path="C:\\Users\\Admin\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe")
    all_links = []

    for page in range(1, max_pages + 1):
        print(f"Đang xử lý trang {page}")
        url = start_url.format(page=page)
        driver.get(url)
        # Thu thập liên kết trên trang hiện tại
        links = get_job_links(driver)
        all_links.extend(links)

    driver.quit()
    return all_links

# URL trang đầu tiên
start_url = "https://careerviet.vn/viec-lam-noi-bat-trong-tuan-l8a30r50p1"

# Thu thập liên kết từ 5 trang đầu tiên
job_links = scrape_multiple_pages(start_url, max_pages=5)

# Lưu kết quả vào CSV
output_file = "job_links.csv"
with open(output_file, "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Job Link"])
    for link in job_links:
        writer.writerow([link])

print(f"Đã lưu {len(job_links)} liên kết vào {output_file}.")


C:\Users\Admin\AppData\Local\Temp\ipykernel_16052\748314238.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="C:\\Users\\Admin\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe")


Đang xử lý trang 1
Đang xử lý trang 2
Đang xử lý trang 3
Đang xử lý trang 4
Đang xử lý trang 5
Đã lưu 500 liên kết vào job_links.csv.


In [ ]:
def get_job_detail(driver):

    # Lấy thông tin công việc
    job_detail = {}
    
    job_name = driver.find_element(By.CLASS_NAME, 'title').text
    print(job_name)
    job_detail['Tên công việc'] = job_name

    map_element = driver.find_element(By.CLASS_NAME, "map")
    location_elements = map_element.find_elements(By.XPATH, './/p/a')
    job_locations = [location.text for location in location_elements]
    print(job_locations)
    job_detail['Địa điểm'] = job_locations

    detail_elements = driver.find_elements(By.CLASS_NAME, 'has-background')
    
    for element in detail_elements:
        li_elements = element.find_elements(By.XPATH, ".//li")
        for li in li_elements:
            # Lấy tiêu đề từ thẻ <strong>
            label = li.find_element(By.XPATH, "./strong").text

            # Lấy giá trị từ thẻ <p>
            value = li.find_element(By.XPATH, "./p").text.strip()
            job_detail[label] = value
    print(job_detail)


    # Lấy thông tin công ty
    tab_element = driver.find_element(By.XPATH, ".//div[@class='tab-content' and @id='tab-2']")
    section_element = tab_element.find_element(By.CLASS_NAME, 'company-profile')



url = 'https://careerviet.vn/vi/tim-viec-lam/nhan-vien-kinh-doanh.35C2A242.html'
driver = webdriver.Chrome(executable_path="C:\\Users\\Admin\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe")
driver.get(url)
get_job_detail(driver)
driver.close()

C:\Users\Admin\AppData\Local\Temp\ipykernel_16740\1558177086.py:35: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="C:\\Users\\Admin\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe")


Nhân viên kinh doanh
['Hồ Chí Minh', 'Long An']
{'Tên công việc': 'Nhân viên kinh doanh', 'Địa điểm': ['Hồ Chí Minh', 'Long An'], 'Ngày cập nhật': '06/12/2024', 'Ngành nghề': 'Vận chuyển / Giao nhận / Kho vận , Hàng hải , Xuất nhập khẩu', 'Hình thức': 'Nhân viên chính thức', 'Lương': 'Cạnh tranh', 'Kinh nghiệm': '2 - 3 Năm', 'Cấp bậc': 'Nhân viên', 'Hết hạn nộp': '28/02/2025'}


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".company-profile"}
  (Session info: chrome=131.0.6778.109)
Stacktrace:
	GetHandleVerifier [0x00007FF79EB96CF5+28821]
	(No symbol) [0x00007FF79EB03880]
	(No symbol) [0x00007FF79E9A578A]
	(No symbol) [0x00007FF79E9F91BE]
	(No symbol) [0x00007FF79E9F94AC]
	(No symbol) [0x00007FF79E9EC52C]
	(No symbol) [0x00007FF79EA1F33F]
	(No symbol) [0x00007FF79E9EC3F6]
	(No symbol) [0x00007FF79EA1F510]
	(No symbol) [0x00007FF79EA3F412]
	(No symbol) [0x00007FF79EA1F0A3]
	(No symbol) [0x00007FF79E9EA778]
	(No symbol) [0x00007FF79E9EB8E1]
	GetHandleVerifier [0x00007FF79EECFCED+3408013]
	GetHandleVerifier [0x00007FF79EEE745F+3504127]
	GetHandleVerifier [0x00007FF79EEDB63D+3455453]
	GetHandleVerifier [0x00007FF79EC5BDFB+835995]
	(No symbol) [0x00007FF79EB0EB9F]
	(No symbol) [0x00007FF79EB0A854]
	(No symbol) [0x00007FF79EB0A9ED]
	(No symbol) [0x00007FF79EAFA1D9]
	BaseThreadInitThunk [0x00007FFED64